
```{raw:latex}
\part{Tema 3. Análisis de Fourier para señales en tiempo continuo}
```

# Tema 3. Análisis de Fourier para señales en tiempo continuo

En el Tema 1 de la asignatura se han caracterizado las señales como distribuciones de una determinada magnitud con respecto a una variable independiente, típicamente el tiempo. Posteriormente, en el Tema 2, se ha abordado la caracterización de los sistemas lineales e invariantes en el tiempo (LTI) en el dominio de esta misma variable, es decir, en el dominio temporal.

*** Figuras

La formalización de la respuesta al impulso de los sistemas LTI ha proporcionado una herramienta de gran utilidad, ya que una única señal, la respuesta al impulso del sistema, permite predecir su comportamiento ante cualquier señal de entrada y, en último término, caracterizar propiedades fundamentales como la estabilidad, la causalidad o la presencia de memoria.

No obstante, la caracterización de señales y sistemas en el dominio temporal se basa en una operación relativamente compleja como es la convolución, lo que dificulta tanto la interpretación cualitativa de la transformación que lleva a cabo un sistema LTI como la predicción directa de la forma de la señal de salida ante una entrada dada.

*** Figura

En el presente tema se introduce una caracterización alternativa en el dominio espectral, o dominio de la frecuencia para señales en tiempo continuo. La ventaja de esta representación es doble:
- Por un lado, la descripción de las señales y los sistemas en términos de su contenido frecuencial conecta de forma natural con fenómenos físicos cotidianos.
- Por otro lado, el efecto de un sistema LTI sobre una señal de entrada se describe en términos de una operación mucho más sencilla que la convolución con la respuesta al impulso $h(t)$: la relación entrada-salida se reduce al producto por una respuesta en frecuencia $H(\omega)$.


### Ejemplo 1:
La diferencia entre dos notas musicales de una misma octava está asociada a su frecuencia o armónico fundamental.

In [35]:
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, TapTool, HoverTool, LabelSet, Div
from bokeh.layouts import column, row

# Configuración silenciosa
output_notebook(verbose=False, hide_banner=True)

# ==========================================
# 1. DEFINICIÓN DE DATOS (Igual que antes)
# ==========================================
notes_data = [
    {'note': 'Do (C4)', 'freq': 261.63, 'x': 1, 'w': 1, 'h': 4, 'type': 'w'},
    {'note': 'Do# (C#4) / Re♭ (D♭4)', 'freq': 277.18, 'x': 1.5, 'w': 0.6, 'h': 2.5, 'type': 'b'},
    {'note': 'Re (D4)', 'freq': 293.66, 'x': 2, 'w': 1, 'h': 4, 'type': 'w'},
    {'note': 'Re# (D#4) / Mi♭ (E♭4)', 'freq': 311.13, 'x': 2.5, 'w': 0.6, 'h': 2.5, 'type': 'b'},
    {'note': 'Mi (E4)', 'freq': 329.63, 'x': 3, 'w': 1, 'h': 4, 'type': 'w'},
    {'note': 'Fa (F4)', 'freq': 349.23, 'x': 4, 'w': 1, 'h': 4, 'type': 'w'},
    {'note': 'Fa# (F#4) / Sol♭ (G♭4)', 'freq': 369.99, 'x': 4.5, 'w': 0.6, 'h': 2.5, 'type': 'b'},
    {'note': 'Sol (G4)', 'freq': 392.00, 'x': 5, 'w': 1, 'h': 4, 'type': 'w'},
    {'note': 'Sol# (G#4) / La♭ (A♭4)', 'freq': 415.30, 'x': 5.5, 'w': 0.6, 'h': 2.5, 'type': 'b'},
    {'note': 'La (A4)', 'freq': 440.00, 'x': 6, 'w': 1, 'h': 4, 'type': 'w'},
    {'note': 'La# (A#4) / Si♭ (B♭4)', 'freq': 466.16, 'x': 6.5, 'w': 0.6, 'h': 2.5, 'type': 'b'},
    {'note': 'Si (B4)', 'freq': 493.88, 'x': 7, 'w': 1, 'h': 4, 'type': 'w'},
]

whites = [n for n in notes_data if n['type'] == 'w']
blacks = [n for n in notes_data if n['type'] == 'b']

source_white = ColumnDataSource(data=dict(
    x=[k['x'] for k in whites], y=[k['h']/2 for k in whites],
    width=[k['w'] for k in whites], height=[k['h'] for k in whites],
    note_name=[k['note'] for k in whites], freq_val=[k['freq'] for k in whites],
    color=['white'] * len(whites)
))

source_black = ColumnDataSource(data=dict(
    x=[k['x'] for k in blacks], y=[k['h']/2 for k in blacks],
    width=[k['w'] for k in blacks], height=[k['h'] for k in blacks],
    note_name=[k['note'] for k in blacks], freq_val=[k['freq'] for k in blacks],
    color=['black'] * len(blacks)
))

# Datos Señales
initial_freq = 261.63
source_spectrum = ColumnDataSource(data=dict(freqs=[initial_freq], amps=[1.0], zeros=[0.0], labels=[f"{initial_freq} Hz"]))
t_max = 0.02 
t_vec = np.linspace(0, t_max, 500)
y_vec = np.sin(2 * np.pi * initial_freq * t_vec)
source_time = ColumnDataSource(data=dict(t=t_vec, y=y_vec))

# ==========================================
# CAMBIO 1: sizing_mode="stretch_width"
# Esto permite que las figuras se adapten al contenedor del Jupyter Book
# ==========================================

# 1. TECLADO
p_keys = figure(height=220, tools="tap", sizing_mode="stretch_width", # Ancho automático
                title="1. Haz clic en una tecla",
                y_range=(4.2, -0.2)) 
p_keys.toolbar_location = None
p_keys.axis.visible = False
p_keys.grid.visible = False
p_keys.outline_line_color = None

renderer_w = p_keys.rect(x='x', y='y', width='width', height='height', 
                         fill_color='color', line_color="gray", source=source_white)
renderer_b = p_keys.rect(x='x', y='y', width='width', height='height', 
                         fill_color='color', line_color="gray", source=source_black)

renderer_w.selection_glyph = renderer_w.glyph.clone()
renderer_w.selection_glyph.fill_color = "#ADD8E6"
renderer_w.nonselection_glyph = renderer_w.glyph
renderer_b.selection_glyph = renderer_b.glyph.clone()
renderer_b.selection_glyph.fill_color = "#ADD8E6"
renderer_b.nonselection_glyph = renderer_b.glyph

tap_tool = p_keys.select(type=TapTool)
p_keys.add_tools(HoverTool(renderers=[renderer_w, renderer_b], tooltips=[("Nota", "@note_name"), ("Frec.", "@freq_val Hz")]))

# 2. TIEMPO (Ancho flexible)
p_time = figure(height=300, title=f"2. Dominio del Tiempo (x(t))", sizing_mode="stretch_width",
                x_axis_label="Tiempo (s)", y_axis_label="Amplitud",
                y_range=(-1.2, 1.2), x_range=(0, t_max))
p_time.line(x='t', y='y', line_width=2, color="#1f77b4", source=source_time)

# 3. FRECUENCIA (Ancho flexible)
p_spec = figure(height=300, title="3. Dominio de la Frecuencia (|X(f)|)", sizing_mode="stretch_width",
                x_axis_label="Frecuencia (Hz)", y_axis_label="Magnitud",
                x_range=(200, 550), y_range=(0, 1.3))
p_spec.segment(x0='freqs', y0='zeros', x1='freqs', y1='amps', line_width=3, color="#d62728", source=source_spectrum)
p_spec.scatter(x='freqs', y='amps', size=10, color="#d62728", fill_color="white", source=source_spectrum)
labels = LabelSet(x='freqs', y='amps', text='labels', level='glyph', x_offset=5, y_offset=5, source=source_spectrum,
                  text_font_size="10pt", text_color="#d62728")
p_spec.add_layout(labels)

# INTERACTIVIDAD (JS)
callback = CustomJS(args=dict(source_w=source_white, source_b=source_black, 
                              source_spec=source_spectrum, source_time=source_time,
                              p_spec=p_spec, p_time=p_time), code="""
    const idx_b = source_b.selected.indices;
    const idx_w = source_w.selected.indices;
    let new_freq = null;
    let new_note = "";
    
    if (idx_b.length > 0) {
        const i = idx_b[0];
        new_freq = source_b.data['freq_val'][i];
        new_note = source_b.data['note_name'][i];
        source_w.selected.indices = []; 
    } else if (idx_w.length > 0) {
        const i = idx_w[0];
        new_freq = source_w.data['freq_val'][i];
        new_note = source_w.data['note_name'][i];
    }
    
    if (new_freq !== null) {
        source_spec.data['freqs'] = [new_freq];
        source_spec.data['labels'] = [new_freq.toFixed(2) + " Hz"];
        p_spec.title.text = "Frecuencia: " + new_freq.toFixed(1) + " Hz (" + new_note + ")";
        source_spec.change.emit();

        const t = source_time.data['t'];
        const y = source_time.data['y'];
        const omega = 2 * Math.PI * new_freq;
        for (let k = 0; k < t.length; k++) { y[k] = Math.sin(omega * t[k]); }
        p_time.title.text = "Tiempo: " + new_note;
        source_time.change.emit();
    }
""")
tap_tool.callback = callback

# ==========================================
# CAMBIO 2: Caption compatible con MODO OSCURO
# Eliminamos 'color: #555' y usamos estilos neutros.
# ==========================================
texto_caption = """
<div style="font-family: sans-serif; margin-top: 10px; font-size: 14px; opacity: 0.8;">
    <b>Ejemplo 1: Notas musicales.</b> Al pulsar las teclas del piano, observa varía el periodo fundamental de la señal en el tiempo (azul), 
    y en frecuencia varía la posición de la frecuencia fundamental (rojo).
    se comprime y el espectro (rojo) se desplaza. La nota se ve mucho mejor en frecuencia que en tiempo.<br>
    <b>Nota:</b> Se muestra una representación simplificada sólo con el modo principal de vibración de las cuerdas del piano, sin incluir sus armónicos.
</div>
"""
caption = Div(text=texto_caption, sizing_mode="stretch_width")

# Layout Responsive
layout = column(p_keys, row(p_time, p_spec, sizing_mode="stretch_width"), caption, sizing_mode="scale_width")

show(layout)

### Ejemplo 2:
El timbre de un instrumento viene determinado por la distinta combinación de armónicos que lo componen; y la coexistencia de múltiples emisoras de radio en un mismo medio físico es posible gracias a la modulación de cada señal en torno a una frecuencia portadora distinta. En este bloque se formalizarán estos conceptos y se mostrará que las señales pueden representarse como superposición de componentes armónicas.
	\item Por otro lado, el efecto de un sistema LTI sobre una señal de entrada se describe en términos de una operación mucho más sencilla que la convolución con la respuesta al impulso $h(t)$: la relación entrada--salida se reduce al producto por una respuesta en frecuencia $H(\omega)$.
\end{itemize}
%

In [45]:
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Select, HoverTool, Div
from bokeh.layouts import column, row

output_notebook(verbose=False, hide_banner=True)

# ==========================================
# 1. PARÁMETROS
# ==========================================
MAX_FREQ_VIEW = 3000
N_HARMONICS = 25

# ==========================================
# 2. DATOS
# ==========================================
def extend_profile(base_profile, length):
    res = list(base_profile)
    if len(res) < length:
        res += [0.0] * (length - len(res))
    return res[:length]

instruments = {
    "Tono Puro": extend_profile([1.0], N_HARMONICS),
    "Clarinete": extend_profile([1.0, 0.1, 0.8, 0.05, 0.6, 0.02, 0.4, 0, 0.2, 0, 0.1], N_HARMONICS),
    "Trompeta":  extend_profile([0.8, 1.0, 0.7, 0.9, 0.5, 0.7, 0.4, 0.5, 0.3, 0.2, 0.15, 0.1, 0.05], N_HARMONICS),
    "Flauta":    extend_profile([1.0, 0.4, 0.1, 0.05, 0.02, 0.01], N_HARMONICS),
    "Sierra":    [1.0/(i+1) for i in range(N_HARMONICS)],
    "Cuadrada":  [1.0/(i+1) if (i+1)%2!=0 else 0 for i in range(N_HARMONICS)]
}

notes_map = {
    "Do grave (C3) - 130 Hz": 130.81,
    "La (A3) - 220 Hz": 220.00,
    "Do central (C4) - 261 Hz": 261.63,
    "Mi (E4) - 330 Hz": 329.63,
    "La (A4) - 440 Hz": 440.00,
    "Do agudo (C5) - 523 Hz": 523.25
}

init_inst = "Clarinete"
init_note = "Do central (C4) - 261 Hz"
init_amps = instruments[init_inst]
f0 = notes_map[init_note]
indices = np.arange(1, N_HARMONICS + 1)

# CDS
freqs_init = indices * f0 
source_freq = ColumnDataSource(data=dict(
    h_idx=indices, freq=freqs_init, amp=init_amps, zeros=np.zeros(N_HARMONICS)
))

t_vec = np.linspace(0, 0.02, 800) 
y_init = np.zeros_like(t_vec)
for i, amp in enumerate(init_amps):
    if amp > 0: y_init += amp * np.sin(2 * np.pi * (i+1) * f0 * t_vec)
if np.max(np.abs(y_init)) > 0: y_init /= np.max(np.abs(y_init))

source_time = ColumnDataSource(data=dict(t=t_vec, y=y_init))

# ==========================================
# 3. GRÁFICAS Y MENÚS
# ==========================================
select_inst = Select(title="Timbre:", value=init_inst, options=list(instruments.keys()), width=200)
select_note = Select(title="Nota:", value=init_note, options=list(notes_map.keys()), width=200)

p_time = figure(height=300, title="Dominio del Tiempo (20 ms)", sizing_mode="stretch_width",
                x_axis_label="Tiempo (s)", y_range=(-1.3, 1.3), x_range=(0, 0.02))
p_time.line(x='t', y='y', line_width=2, color="#1f77b4", source=source_time)
p_time.grid.grid_line_alpha = 0.3

p_freq = figure(height=300, title=f"Espectro (límite {MAX_FREQ_VIEW} Hz)", sizing_mode="stretch_width",
                x_axis_label="Frecuencia (Hz)", y_axis_label="Amplitud",
                x_range=(0, MAX_FREQ_VIEW), y_range=(0, 1.2))

p_freq.segment(x0='freq', y0='zeros', x1='freq', y1='amp', line_width=3, color="#d62728", source=source_freq)
p_freq.scatter(x='freq', y='amp', size=8, color="#d62728", fill_color="white", line_width=2, source=source_freq)
p_freq.line(x='freq', y='amp', line_width=2, color="#d62728", line_dash="dashed", alpha=0.5, source=source_freq)
p_freq.add_tools(HoverTool(tooltips=[("n", "@h_idx"), ("f", "@freq{0.} Hz"), ("Amp", "@amp{0.00}")], mode='vline'))

# ==========================================
# 4. JAVASCRIPT CORREGIDO
# ==========================================
# Eliminamos el %d del string y pasamos N en 'args'
js_code = """
    const inst_name = select_inst.value;
    const note_name = select_note.value;
    
    // N ahora viene directamente de los argumentos (args), no hace falta formatear string
    
    const instruments_db = {
        "Tono Puro": [], "Clarinete": [], "Trompeta":  [], "Flauta":    [],
    };
    
    function fill(arr, vals) {
        for(let i=0; i<N; i++) {
            if(i < vals.length) arr.push(vals[i]);
            else arr.push(0.0);
        }
    }
    
    fill(instruments_db["Tono Puro"], [1.0]);
    fill(instruments_db["Clarinete"], [1.0, 0.1, 0.8, 0.05, 0.6, 0.02, 0.4, 0, 0.2, 0, 0.1]);
    fill(instruments_db["Trompeta"],  [0.8, 1.0, 0.7, 0.9, 0.5, 0.7, 0.4, 0.5, 0.3, 0.2, 0.15, 0.1]);
    fill(instruments_db["Flauta"],    [1.0, 0.4, 0.1, 0.05, 0.02, 0.01]);
    
    instruments_db["Sierra"] = [];
    instruments_db["Cuadrada"] = [];
    for(let i=1; i<=N; i++) {
        instruments_db["Sierra"].push(1.0/i);
        // Aquí estaba el problema: el % se confundía con Python. Ahora es seguro.
        if (i % 2 !== 0) instruments_db["Cuadrada"].push(1.0/i);
        else instruments_db["Cuadrada"].push(0.0);
    }
    
    const notes_db = {
        "Do grave (C3) - 130 Hz": 130.81,
        "La (A3) - 220 Hz": 220.00,
        "Do central (C4) - 261 Hz": 261.63,
        "Mi (E4) - 330 Hz": 329.63,
        "La (A4) - 440 Hz": 440.00,
        "Do agudo (C5) - 523 Hz": 523.25
    };

    const new_amps = instruments_db[inst_name];
    const f0 = notes_db[note_name];
    
    if (!new_amps || !f0) return;
    
    // UPDATE FRECUENCIA
    const freqs_array = source_freq.data['freq'];
    const amps_array = source_freq.data['amp'];
    
    for (let i = 0; i < N; i++) {
        freqs_array[i] = (i + 1) * f0; 
        amps_array[i] = new_amps[i];
    }
    source_freq.change.emit();
    
    // UPDATE TIEMPO
    const t = source_time.data['t'];
    const y = source_time.data['y'];
    const omega0 = 2 * Math.PI * f0;
    
    let max_val = 0;
    for (let i = 0; i < t.length; i++) {
        let sum = 0;
        for (let h = 0; h < N; h++) {
             const amp = new_amps[h];
             if (amp > 0) {
                 sum += amp * Math.sin((h + 1) * omega0 * t[i]);
             }
        }
        y[i] = sum;
        if (Math.abs(sum) > max_val) max_val = Math.abs(sum);
    }
    
    if (max_val > 0.0001) {
         for (let i = 0; i < t.length; i++) { y[i] = y[i] / max_val; }
    }
    source_time.change.emit();
""" 

# CAMBIO CLAVE AQUÍ: Pasamos N como argumento en el diccionario args
callback = CustomJS(args=dict(source_freq=source_freq, source_time=source_time, 
                              select_inst=select_inst, select_note=select_note,
                              N=N_HARMONICS), # <--- AQUÍ PASAMOS LA VARIABLE PYTHON A JS
                    code=js_code)

select_inst.js_on_change('value', callback)
select_note.js_on_change('value', callback)

# ==========================================
# 5. LAYOUT
# ==========================================
texto_caption = """
<div style="font-family: sans-serif; margin-top: 10px; font-size: 13px; opacity: 0.8;">
    <b>Sintetizador de Fourier:</b>
    Selecciona un instrumento y una nota. La gráfica de la derecha muestra todos los armónicos (hasta 25) 
    que caben en el rango visualizado.
</div>
"""
caption = Div(text=texto_caption, sizing_mode="stretch_width")

widgets_row = row(select_inst, select_note)
layout = column(widgets_row, row(p_time, p_freq, sizing_mode="stretch_width"), caption, sizing_mode="scale_width")

show(layout)